# Project 3 - Sagittarius A* Exploration
### By Angela Less & Steven Dahms

## Project Goal
The goal of this project was to take the reported orbital data of star S2 and fit an elipse to it's orbit around Sag A*, the supermassive black hole at the center of our galaxy. This experiment won a Nobel Prize for Physics in 2020, for Andrea Ghez and Reinhard Genzel. While Sag A* has been known of since the 1930's, it wasn't until this experiment that they conclusively proved that it could only be a black hole, which was then subsequently photographed in 2022.

## So How Did We Do It?
If this work is winning a Nobel Prize, how did two undergrads manage to do it? A mix of youthful vigor and caffeine mostly, but we also have the power of published data and confirmation bias on our side.

We arranged our code into 3 files. 1 python script for data handling, and two streamlit apps, each with their own purposes.

## Python Script
This first block of code queries SIMBAD for the data on Sag A*, which we use later on in the streamlit app. The main thing that we're extracting is the positional data, for now.

In [ ]:
def get_sagittarius_a_data():
    """
    Retrieve data for Sagittarius A* (Sgr A*), the supermassive black hole at the Galactic Center.

    Returns:
    --------
    pandas.DataFrame : SIMBAD data for Sgr A*
    dict : Additional properties extracted from the data
    """
    print("Querying SIMBAD for Sagittarius A*...")

    # Customize Simbad to return useful columns
    custom_simbad = Simbad()
    custom_simbad.add_votable_fields('otype', 'sp', 'flux(V)', 'flux(B)', 'plx', 'pm', 'rv', 'z_value')

    try:
        # Query Sagittarius A*
        result_table = custom_simbad.query_object("Sagittarius A*")

        if result_table is not None and len(result_table) > 0:
            df = result_table.to_pandas()

            # Extract key properties
            properties = {
                'name': df['MAIN_ID'].iloc[0] if 'MAIN_ID' in df.columns else 'Sagittarius A*',
                'ra': df['RA'].iloc[0] if 'RA' in df.columns else None,
                'dec': df['DEC'].iloc[0] if 'DEC' in df.columns else None,
                'object_type': df['OTYPE'].iloc[0] if 'OTYPE' in df.columns else None,
            }

            print(f"Found data for {properties['name']}")
            return df, properties
        else:
            print("No data found for Sagittarius A*")
            return pd.DataFrame(), {}

    except Exception as e:
        print(f"Error querying SIMBAD: {e}")
        return pd.DataFrame(), {}

The second block does the same thing for our reference point S2. This time we take a little more information, like proper motion and radial velocity.

To note: We have to do a fair bit of error handling on this one, just in case we're not able to find it easily. We search by name, and then co-ordinates if that doesn't work. Then we have to trim out the excess entries.

In [ ]:
def get_s2_star_data():
    """
    Retrieve data for star S2 (also known as S0-2), which orbits Sgr A*.

    Returns:
    --------
    pandas.DataFrame : SIMBAD data for S2
    dict : Additional properties
    """
    print("Querying SIMBAD for star S2 (S0-2)...")

    custom_simbad = Simbad()
    custom_simbad.add_votable_fields('otype', 'sp', 'flux(V)', 'flux(B)', 'plx', 'pm', 'rv', 'z_value')

    try:
        # Try different names for S2
        names_to_try = ['S2', 'S0-2', 'S0 2', 'S02', 'Sgr A* S2']

        result_table = None
        for name in names_to_try:
            try:
                result_table = custom_simbad.query_object(name)
                if result_table is not None and len(result_table) > 0:
                    print(f"Found S2 using name: {name}")
                    break
            except:
                continue

        if result_table is None or len(result_table) == 0:
            # Query by coordinates near Sgr A*
            sgr_a_coords = (266.4168, -29.0078)  # Sgr A* coordinates in degrees
            result_table = custom_simbad.query_region(
                f"{sgr_a_coords[0]} {sgr_a_coords[1]}",
                radius="0d1m0s"  # 1 arcminute radius
            )

        if result_table is not None and len(result_table) > 0:
            df = result_table.to_pandas()

            # Filter for S2 if multiple results (look for S2 in the name)
            if len(df) > 1:
                s2_mask = df['MAIN_ID'].str.contains('S2|S0-2|S02', case=False, na=False)
                if s2_mask.any():
                    df = df[s2_mask]

            properties = {
                'name': df['MAIN_ID'].iloc[0] if 'MAIN_ID' in df.columns else 'S2',
                'ra': df['RA'].iloc[0] if 'RA' in df.columns else None,
                'dec': df['DEC'].iloc[0] if 'DEC' in df.columns else None,
                'spectral_type': df['SP_TYPE'].iloc[0] if 'SP_TYPE' in df.columns else None,
                'proper_motion_ra': df['PMRA'].iloc[0] if 'PMRA' in df.columns else None,
                'proper_motion_dec': df['PMDEC'].iloc[0] if 'PMDEC' in df.columns else None,
                'radial_velocity': df['RV_VALUE'].iloc[0] if 'RV_VALUE' in df.columns else None,
            }

            print(f"Found data for {properties['name']}")
            return df, properties
        else:
            print("No data found for S2")
            return pd.DataFrame(), {}

    except Exception as e:
        print(f"Error querying SIMBAD: {e}")
        return pd.DataFrame(), {}

This function searches near Sag A* and compiles data on stars near it. Only a radius of 2 arcminutes, as can be seen.

In [ ]:
def get_s2_orbital_data_from_vizier():
    """
    Query Vizier catalogs for published S2 orbital data and parameters.
    Searches for catalogs containing orbital elements and observational data.

    Returns:
    --------
    pandas.DataFrame : Orbital parameters and observational data
    """
    print("Querying Vizier for S2 orbital data...")

    # Sgr A* coordinates
    sgr_a_coords = (266.4168, -29.0078)

    try:
        v = Vizier(columns=['**'], row_limit=500)

        # Search in a small region around Sgr A*
        result_list = v.query_region(
            f"{sgr_a_coords[0]} {sgr_a_coords[1]}",
            radius="0d2m0s",  # 2 arcminute radius
            catalog=["J/ApJ/*", "J/A+A/*", "J/MNRAS/*"]  # Common astronomy journals
        )

        if result_list:
            # Combine all results
            all_data = []
            for table in result_list:
                df = table.to_pandas()
                all_data.append(df)

            if all_data:
                combined_df = pd.concat(all_data, ignore_index=True)
                print(f"Found {len(combined_df)} records from Vizier")
                return combined_df

        return pd.DataFrame()

    except Exception as e:
        print(f"Error querying Vizier: {e}")
        return pd.DataFrame()

We combine that with data from the following method...

In [ ]:
def get_gaia_data_for_region(ra=266.4168, dec=-29.0078, radius_arcsec=30):
    """
    Query Gaia catalog for stellar data in the region around Sgr A*.
    This can provide proper motions and positions for stars in the region.

    Parameters:
    -----------
    ra : float
        Right ascension in degrees
    dec : float
        Declination in degrees
    radius_arcsec : float
        Search radius in arcseconds

    Returns:
    --------
    pandas.DataFrame : Gaia catalog data
    """
    print(f"Querying Gaia for stellar data within {radius_arcsec} arcsec of Sgr A*...")

    try:
        # Query Gaia DR3
        job = Gaia.launch_job_async(
            f"""
            SELECT TOP 1000
                source_id, ra, dec,
                parallax, parallax_error,
                pmra, pmra_error, pmdec, pmdec_error,
                phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag,
                radial_velocity, radial_velocity_error,
                astrometric_excess_noise
            FROM gaiadr3.gaia_source
            WHERE 1=CONTAINS(
                POINT('ICRS', ra, dec),
                CIRCLE('ICRS', {ra}, {dec}, {radius_arcsec/3600.0})
            )
            AND parallax IS NOT NULL
            ORDER BY phot_g_mean_mag
            """
        )

        result_table = job.get_results()

        if result_table is not None and len(result_table) > 0:
            df = result_table.to_pandas()
            print(f"Found {len(df)} stars in Gaia catalog")
            return df
        else:
            print("No Gaia data found")
            return pd.DataFrame()

    except Exception as e:
        print(f"Error querying Gaia: {e}")
        return pd.DataFrame()

This runs a query on the Gaia catalog to get more data from the surrounding region. So now we have two data sources queried for information on the neighborhood of SagA*.

Finally we query Simbad to give us one last source.

In [ ]:
def get_galactic_center_stars():
    """
    Query for multiple stars in the Galactic Center region that orbit Sgr A*.
    This includes the S-stars cluster.

    Returns:
    --------
    pandas.DataFrame : Data for Galactic Center stars
    """
    print("Querying for Galactic Center stars...")

    sgr_a_coords = (266.4168, -29.0078)

    try:
        custom_simbad = Simbad()
        custom_simbad.add_votable_fields('otype', 'sp', 'pm', 'rv')

        # Query region around Sgr A*
        result_table = custom_simbad.query_region(
            f"{sgr_a_coords[0]} {sgr_a_coords[1]}",
            radius="0d2m0s"  # 2 arcminute radius
        )

        if result_table is not None and len(result_table) > 0:
            df = result_table.to_pandas()

            # Filter for stars (remove extended sources, galaxies, etc.)
            if 'OTYPE' in df.columns:
                star_mask = ~df['OTYPE'].str.contains('G|Neb|HII|PN', case=False, na=False)
                df = df[star_mask]

            print(f"Found {len(df)} stars in the Galactic Center region")
            return df
        else:
            return pd.DataFrame()

    except Exception as e:
        print(f"Error querying for Galactic Center stars: {e}")
        return pd.DataFrame()

Then we compile the data...

In [ ]:
def compile_all_data():
    """
    Compile all available data for Sgr A* and S2.
    This is the main function to call for getting all relevant data.

    Returns:
    --------
    dict : Dictionary containing all retrieved data
    """
    print("=" * 70)
    print("Compiling data for Sagittarius A* and Star S2 (S0-2)")
    print("=" * 70)

    all_data = {}

    # Get Sgr A* data
    sgr_a_df, sgr_a_props = get_sagittarius_a_data()
    all_data['sgr_a_star'] = sgr_a_df
    all_data['sgr_a_properties'] = sgr_a_props

    # Get S2 star data
    s2_df, s2_props = get_s2_star_data()
    all_data['s2_star'] = s2_df
    all_data['s2_properties'] = s2_props

    # Get S2 orbital data from Vizier
    s2_orbital = get_s2_orbital_data_from_vizier()
    all_data['s2_orbital_data'] = s2_orbital

    # Get Gaia data for the region
    gaia_data = get_gaia_data_for_region()
    all_data['gaia_data'] = gaia_data

    # Get other Galactic Center stars
    gc_stars = get_galactic_center_stars()
    all_data['galactic_center_stars'] = gc_stars

    print("\n" + "=" * 70)
    print("Data compilation complete!")
    print("=" * 70)

    return all_data

And save everything to a data file.

In [ ]:
def save_data_to_files(data_dict, prefix="sgr_a_s2_data"):
    """
    Save all retrieved data to CSV files.

    Parameters:
    -----------
    data_dict : dict
        Dictionary containing dataframes from compile_all_data()
    prefix : str
        Prefix for output filenames
    """
    import os

    for key, value in data_dict.items():
        if isinstance(value, pd.DataFrame) and not value.empty:
            filename = f"{prefix}_{key}.csv"
            value.to_csv(filename, index=False)
            print(f"Saved {key} to {filename}")
        elif isinstance(value, dict):
            # Save properties as a simple text file
            filename = f"{prefix}_{key}.txt"
            with open(filename, 'w') as f:
                for k, v in value.items():
                    f.write(f"{k}: {v}\n")
            print(f"Saved {key} to {filename}")


# Example usage and main execution
if __name__ == "__main__":
    print("\n" + "=" * 70)
    print("Sagittarius A* and Star S2 (S0-2) Data Retrieval")
    print("=" * 70 + "\n")

    # Compile all data
    data = compile_all_data()

    # Display summary
    print("\n" + "=" * 70)
    print("DATA SUMMARY")
    print("=" * 70)

    if not data['sgr_a_star'].empty:
        print("\nSagittarius A* Data:")
        print(data['sgr_a_star'][['MAIN_ID', 'RA', 'DEC', 'OTYPE']].to_string())

    if not data['s2_star'].empty:
        print("\nS2 Star Data:")
        print(data['s2_star'][['MAIN_ID', 'RA', 'DEC', 'SP_TYPE']].to_string())

    if not data['s2_orbital_data'].empty:
        print(f"\nS2 Orbital Data: {len(data['s2_orbital_data'])} records")
        print("Columns:", list(data['s2_orbital_data'].columns))

    if not data['gaia_data'].empty:
        print(f"\nGaia Data: {len(data['gaia_data'])} stars")
        print("Sample columns:", list(data['gaia_data'].columns[:10]))

    if not data['galactic_center_stars'].empty:
        print(f"\nGalactic Center Stars: {len(data['galactic_center_stars'])} stars")

    # Optionally save to files
    # save_data_to_files(data)

    print("\n" + "=" * 70)
    print("Data retrieval complete!")
    print("=" * 70)

## Streamlit App 1: Black Hole Teaching Tool
This first streamlit app simply takes in a user defined stellar mass, and creates a 3D graph of a black hole, and the size of it's accretion disk.

This block calculates the properties of the black hole from the user-defined mass.

In [ ]:
# Mass input
mass_solar = st.sidebar.number_input(
    "Mass (Solar Masses)",
    min_value=0.1,
    max_value=1000000.0,
    value=10.0,
    step=0.1,
    help="Mass of the black hole in solar masses"
)
st.sidebar.markdown(
    "**Examples:**  \n"
    "- Typical star: 1 M☉  \n"
    "- Massive star: 20 M☉  \n"
    "- Stellar black hole: 50 M☉  \n"
    "- Intermediate BH: 10⁴ M☉  \n"
    "- Sagittarius A*: 4.3×10⁶ M☉"
)

# Calculate properties
solar_mass = 1.989e30  # kg
mass_kg = mass_solar * solar_mass

# Schwarzschild radius
schwarzschild_radius = (2 * G * mass_kg) / (c ** 2)
schwarzschild_radius_km = schwarzschild_radius / 1000
schwarzschild_radius_miles = schwarzschild_radius_km * 0.621371

size_comparisons = [
    (0.1, "about the length of a football field"),
    (1, "roughly the height of the Empire State Building"),
    (5, "similar to the width of Manhattan"),
    (16, "close to the width of the Grand Canyon"),
    (100, "about the distance across Los Angeles"),
    (6371, "comparable to Earth's radius"),
    (696340, "approaching the Sun's radius"),
]

relative_size = "far larger than the Sun's radius"
for threshold, description in size_comparisons:
    if schwarzschild_radius_km <= threshold:
        relative_size = description
        break

# Event horizon area
event_horizon_area = 4 * np.pi * schwarzschild_radius ** 2

# Hawking temperature (simplified)
hawking_temp = (6.17e-8) / mass_solar  # Kelvin

# Main content area
col1, col2 = st.columns(2)

And this block plots out the black hole with the measurements we calculated above.

In [ ]:
with col2:
    st.header("Visualization")

    # Create a 3D visualization
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection="3d")

    # Event horizon sphere
    u = np.linspace(0, 2 * np.pi, 60)
    v = np.linspace(0, np.pi, 30)
    x = schwarzschild_radius_km * np.outer(np.cos(u), np.sin(v))
    y = schwarzschild_radius_km * np.outer(np.sin(u), np.sin(v))
    z = schwarzschild_radius_km * np.outer(np.ones_like(u), np.cos(v))
    ax.plot_surface(x, y, z, color="black", alpha=0.8, linewidth=0, shade=True)

    # Accretion disk (thin torus approximation)
    disk_r_outer = schwarzschild_radius_km * 3
    disk_r_inner = schwarzschild_radius_km * 1.2
    disk_u = np.linspace(0, 2 * np.pi, 200)
    disk_v = np.linspace(disk_r_inner, disk_r_outer, 2)
    disk_u, disk_v = np.meshgrid(disk_u, disk_v)
    disk_x = disk_v * np.cos(disk_u)
    disk_y = disk_v * np.sin(disk_u)
    disk_z = np.zeros_like(disk_x)
    ax.plot_surface(
        disk_x, disk_y, disk_z, color="orange", alpha=0.3, linewidth=0, shade=False
    )

    limit = disk_r_outer * 1.2
    ax.set_xlim(-limit, limit)
    ax.set_ylim(-limit, limit)
    ax.set_zlim(-limit, limit)
    ax.set_xlabel("X (km)")
    ax.set_ylabel("Y (km)")
    ax.set_zlabel("Z (km)")
    ax.set_title("Event Horizon & Accretion Disk")
    ax.view_init(elev=25, azim=35)

    st.pyplot(fig)
    st.caption(
        f"This event horizon radius is {relative_size}, making the entire diameter "
        f"≈ {2 * schwarzschild_radius_km:.2f} km."
    )

All things considered, it's actually rather simple. Most of the lines come from the fact that there's simply more data required in plotting a 3D object.